In [3]:
# import torch
# import pickle
# from models.experimental import attempt_load  # YOLOv7 모델 로더

# # 1. 모델 로드 (YOLOv7)
# weights_path = 'runs/train/rcs-yolo/weights/best.pt'
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = attempt_load(weights_path, map_location=device)
# model.eval()

# # 2. Pickle로 저장
# with open('./brainMRI_model_weights.pkl', 'wb') as f:
#     pickle.dump(model, f)

Fusing layers... 
IDetect.fuse


# Brain MRI

In [9]:
import torch
import pickle
import cv2
import numpy as np

from utils.general import non_max_suppression, scale_coords
from utils.datasets import letterbox

# 1. 모델 불러오기
with open('./brainMRI_model_weights.pkl', 'rb') as f:
    model = pickle.load(f)
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 2. 이미지 로드 및 전처리
img_path = './y500.jpg'
img0 = cv2.imread(img_path)  # BGR
img = letterbox(img0, new_shape=640)[0]
img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR->RGB->CHW
img = np.ascontiguousarray(img)
img = torch.from_numpy(img).float().to(device) / 255.0
img = img.unsqueeze(0)  # batch dimension

# 3. Inference
with torch.no_grad():
    pred = model(img, augment=False)[0]
    pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45)

# 4. 결과 출력
for det in pred:
    if len(det):
        det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()
        for *xyxy, conf, cls in det:
            print(int(cls), "confidence score: ", float(conf.cpu()), "coordinate:", [int(b.cpu()) for b in xyxy])

0 0.8493150472640991 [4, 9, 24, 34]
